# Notebook for Solar Wind Exploration

In the initial phase, we want to see if we can detect FTEs using unsupervised learning, by finding a manifold for the solar wind data.

The initial hypothesis is the transition matrices (Markov Matrices $M$) that can be derived from Manifolder + clustering will show distinctive clusters and transitions.  We can check accuracy by looking at the label (FTE or not?), and see if this label could have been deduced from the data itself.



In [1]:
# useful set of python includes

%load_ext autoreload
%autoreload 2

import numpy as np

np.set_printoptions(suppress=True, precision=4)

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

import seaborn as sns
sns.set()

import pandas as pd

import time

import random

### Load Solar Wind Data, and Run Manifolder

The `dataset_2` file contains 

Dataset-2 (THEMIS):   a list with FTEs periods and non-FTEs periods observed by THEMIS in 2007.  These are combined into one file, randomly FTE - NonFTE - FTE - FTE, NonFTE, etc…

In total there are 63 FTEs and 47 non-FTEs.

The time series are separated by one blank line, and each one has 1440 points in a period of 6 minutes.


In [9]:
#import sys
#sys.path.append(r"your\path\here")

import manifolder as mr
from manifolder import helper as mh


In [3]:
# load the data
# note, you must have started the notebook in the 

print('loading data ...')
df = pd.read_excel('astro_data/dataset_2.xlsx', index_col=0)
df.head()


loading data ...


,bx,by,bz,bl,bm,bn,bmag,vx,vy,vz,vmag,np,Unnamed: 13,tpar,tper,tp,goal
sc time,,,,,,,,,,,,,,,,,
thd,1.177999e+09,2.06,-4.87,17.77,17.80,4.55,-2.48,18.54,-5.96,2.42,6.94,9.46,0.11,3938.25,3565.04,3689.44,1.0
thd,1.177999e+09,2.01,-4.89,17.69,17.72,4.53,-2.53,18.47,-7.04,1.11,6.79,9.85,0.11,3924.28,3610.40,3715.03,1.0
thd,1.177999e+09,2.01,-4.87,17.71,17.74,4.51,-2.52,18.48,-8.14,-0.21,6.66,10.52,0.11,3910.35,3656.06,3740.82,1.0
thd,1.177999e+09,2.02,-4.80,17.74,17.77,4.47,-2.45,18.48,-9.23,-1.53,6.52,11.40,0.11,3896.40,3701.56,3766.51,1.0
thd,1.177999e+09,1.99,-4.83,17.70,17.73,4.47,-2.50,18.45,-10.26,-2.82,6.37,12.40,0.11,3882.37,3746.46,3791.77,1.0


In [5]:
# convert values from loaded spreadsheet, into a numpy matrices
# note that there is no need for the first value, which is time,
# as it is not part of the manifold
#
# also, note the spreadsheet is missing a column name for `Unnamed: 13`, and the values above
# this have the incorrect column labels; the first relevant vale is bx, which as a magnitude around 2
#
# note the final value of each row is the goal (0 or 1), and not part of z

data_raw = df.values[:, 1:]
print('first line of raw_data:\n', data_raw[0, :])


first line of raw_data:
 [   2.06   -4.87   17.77   17.8     4.55   -2.48   18.54   -5.96    2.42
    6.94    9.46    0.11 3938.25 3565.04 3689.44    1.  ]


In [6]:
# loop through the data, breaking out the clusters
# i will always point to the NaN (blank line) in the dataframe,
# and values [i-1440:i] is the snipped

snippet_len = 1440

# callect the snippets into two groups, one for each goal (target) value, 0 or 1
# these can be easily merged
zs_0 = []
zs_1 = []

df.values[0,:]

for i in range(snippet_len,data_raw.shape[0],snippet_len+1):
    # copy the snipped, excluding the last value, which is the goal
    snippet = data_raw[i-snippet_len:i,:-1]

    # grab the goal value from the first row of each snippet
    goal = data_raw[i-snippet_len,-1]
    
    # check to make sure each snippet does not contain NaN
    # (should not, if parsing is correct)
    assert ~np.isnan(snippet).any(), 'oops, snippet contains a Nan!'
    
    print('snippet size',snippet.shape,'with goal',goal)
    
    if goal == 0:
        zs_0.append( snippet )
    elif goal == 1:
        zs_1.append( snippet )
    else:
        assert False, 'value of goal not understood'

# shuffle this lists; this should not strictly be necessary, if all the data is being used,
# but prevents biases when shortening the list

random.shuffle(zs_0)
random.shuffle(zs_1)

shorten_data = False

if shorten_data:
    zs_0 = zs_0[:10]
    zs_1 = zs_1[:10]
        
zs = zs_0 + zs_1

print( '\done!')
print( '\t len(zs_0):',len(zs_0))
print( '\t len(zs_1):',len(zs_1))
print( '\t len(zs):',len(zs))


snippet size (1440, 15) with goal 1.0
snippet size (1440, 15) with goal 0.0
snippet size (1440, 15) with goal 1.0
snippet size (1440, 15) with goal 0.0
snippet size (1440, 15) with goal 1.0
snippet size (1440, 15) with goal 1.0
snippet size (1440, 15) with goal 0.0
snippet size (1440, 15) with goal 1.0
snippet size (1440, 15) with goal 0.0
snippet size (1440, 15) with goal 0.0
snippet size (1440, 15) with goal 1.0
snippet size (1440, 15) with goal 0.0
snippet size (1440, 15) with goal 1.0
snippet size (1440, 15) with goal 0.0
snippet size (1440, 15) with goal 1.0
snippet size (1440, 15) with goal 0.0
snippet size (1440, 15) with goal 0.0
snippet size (1440, 15) with goal 0.0
snippet size (1440, 15) with goal 1.0
snippet size (1440, 15) with goal 0.0
snippet size (1440, 15) with goal 0.0
snippet size (1440, 15) with goal 1.0
snippet size (1440, 15) with goal 0.0
snippet size (1440, 15) with goal 1.0
snippet size (1440, 15) with goal 0.0
snippet size (1440, 15) with goal 1.0
snippet size

In [10]:
# data has been parsed, now run Manifolder

start_time = time.time()

# create manifolder object
# note that with these parameters, the 'computing local cov' step can have a singularity error!
manifolder = mr.Manifolder(nbins=10, ncov=10)

# add the data, and fit (this runs all the functions)
manifolder.fit_transform(zs)

elapsed_time = time.time() - start_time
print('\n\t Program Executed in', str(np.round(elapsed_time, 2)), 'seconds')  # about 215 seconds (four minutes)


calculating histograms for snip  0  of  103  (dim  15  timeseries) ............... done
calculating histograms for snip  1  of  103  (dim  15  timeseries) ............... done
calculating histograms for snip  2  of  103  (dim  15  timeseries) ............... done
calculating histograms for snip  3  of  103  (dim  15  timeseries) ............... done
calculating histograms for snip  4  of  103  (dim  15  timeseries) ............... done
calculating histograms for snip  5  of  103  (dim  15  timeseries) ............... done
calculating histograms for snip  6  of  103  (dim  15  timeseries) ............... done
calculating histograms for snip  7  of  103  (dim  15  timeseries) ............... done
calculating histograms for snip  8  of  103  (dim  15  timeseries) ............... done
calculating histograms for snip  9  of  103  (dim  15  timeseries) ............... done
calculating histograms for snip  10  of  103  (dim  15  timeseries) ............

KeyboardInterrupt: 

Need to find a parameters to run the `fit_transform`?

Getting various errors.  With 10 + 10 snippets, running 

```python
manifolder = mr.Manifolder(nbins=10, ncov=10)
manifolder.fit_transform(zs)
```

gives:

```
~/Anaconda3/anaconda3/lib/python3.7/site-packages/manifolder/main.py in _embedding(self)
    376 
    377         # [V, E] = eigs(W2, 10) Matlab
--> 378         V, E = mh.eigs_like_matlab(W2, 10)  # think this is correct now ...
    379 
    380         # print('V.shape', V.shape)

LinAlgError: Array must not contain infs or NaNs
```


### TODO:

After running `fit_transform()`, use kmeans to label clusters withing all the snippets

Create a transition matrix for each snippet; the zs_0 and zs_1 should have distincively different matrices, which can be used to categorize the snippet



In [ ]:
### _cluster() function, local to make it easier to work on
### ... note, all the individual clusters should be marked invidually?
### but the original kmeans run run all of them together?
###

# Configuration
numClusters = 7  # NOTE, this was previously 14 (too many!)
intrinsicDim = manifolde.Dim  # can be varied slightly but shouldn't be much larger than Dim

## Clusters
# IDX = kmeans(Psi(:, 1:intrinsicDim), numClusters)

# Python kmeans see
# https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.cluster.vq.kmeans.html
# scipy.cluster.vq.kmeans(obs, k_or_guess, iter=20, thresh=1e-05)
#
#  note, python expects each ROW to be an observation, looks the same a matlap
#

print('running k-means')

kmeans = KMeans(n_clusters=numClusters).fit(manifolder.Psi[:, :intrinsicDim])
IDX = kmeans.labels_

# TODO decide how to plot multiple snips
# think that x_ref[1,:] is just
for snip in range(len(self.z)):
    if snip == 0:
        x = self.z[snip][0, :]
        xref1 = x[::self.stepSize]  # downsample, to match the data steps
    else:
        x = self.z[snip][0, :]
        x = x[::self.stepSize]
        xref1 = np.append(xref1, x)

print(xref1.shape)

xs = manifolder.Psi[:, 0]
ys = manifolder.Psi[:, 1]
zs = manifolder.Psi[:, 2]

# normalize these to amplitude one?
print('normalizing amplitudes of Psi in Python ...')
xs /= np.max(np.abs(xs))
ys /= np.max(np.abs(ys))
zs /= np.max(np.abs(zs))

# xs -= np.mean(xs)
# ys -= np.mean(ys)
# zs -= np.mean(zs)

# xs /= np.std(xs)
# ys /= np.std(ys)
# zs /= np.std(zs)

print(xs.shape)

lim = 2000
val = xref1[:lim]
idx = manifolder.IDX[:lim]

plt.figure(figsize=[15, 3])

plt.plot(xref1[:lim], color='black', label='Timeseries')
# plt.plot(xs[:lim], linewidth=.5, label='$\psi_0$')
# plt.plot(ys[:lim], linewidth=.5, label='$\psi_1$')
# plt.plot(zs[:lim], linewidth=.5, label='$\psi_2$')

plt.plot(xs[:lim], linewidth=.5, label='psi_0')
plt.plot(ys[:lim], linewidth=.5, label='psi_1')
plt.plot(zs[:lim], linewidth=.5, label='psi_2')

plt.plot(idx / np.max(idx) + 1, linewidth=.8, label='IDX')

plt.legend()

# rightarrow causes an image error, when displayed in github!
# plt.xlabel('Time $ \\rightarrow $')
plt.xlabel('Time')
plt.ylabel('Value')

# plt.gca().autoscale(enable=True, axis='both', tight=None )
# plt.gca().xaxis.set_ticklabels([])
# plt.gca().yaxis.set_ticklabels([])

plt.title('Example Timeseries and Manifold Projection')

print('done')

###
### additional parsing, for color graphs
###
import matplotlib

cmap = matplotlib.cm.get_cmap('Spectral')

r = xs[:lim]
g = ys[:lim]
b = zs[:lim]

# prevent the jump in data value
r[:self.H] = r[self.H]
g[:self.H] = g[self.H]
b[:self.H] = b[self.H]

r -= np.min(r)
r /= np.max(r)

g -= np.min(g)
g /= np.max(g)

b -= np.min(b)
b /= np.max(b)

plt.figure(figsize=[15, 3])

for i in range(lim - 1):
    col = [r[i], g[i], b[i]]
    plt.plot([i, i + 1], [val[i], val[i + 1]], color=col)

plt.title('data, colored according to Psi (color three-vector)')
plt.xlabel('Time')
plt.ylabel('Value')

plt.show()


In [ ]:
# clustering data ...

IDX = manifolder.IDX

cluster_lens = mh.count_cluster_lengths(IDX)

# cluster_lens is a dictionary a dictonary, where each key is the cluster number (0:6),
# and the values are a list of clusner lengths

mh.show_cluster_lens(cluster_lens)


### Graph Transition (Markov) Matrix

The system can be though of as in one particular "state" (cluster value) at any given time.  This state $S$ can be though of as a column vector with $C$ dimensions, similar to states in quantum mechanic, where the column vector plays the role of the transition matrix.

Time evolution is this given by the tranistion matrix $M$, which is a Markov matrix (all columns sum to one, to preserve probability).  In this case, we have

$$
S_{n+1} = M @ S_n 
$$

Where the $@$ symbol is used to explicitly denote matrix multiplication.

Since most clusters with transition to themselves, the diagonal values of the matrix can be quite high, and are typically removed.  Thus, for visualization, we remove the diagonal elements of the matrix.

In [ ]:
# in this case, index goes from 0 to 6 ... 
# can also have outlier groups in kmeans, need to check for this

print(IDX.shape)
print(np.min(IDX))
print(np.max(IDX))

IDX_max = np.max(IDX)


In [ ]:
M = mh.make_transition_matrix(IDX)
print('\n transition matrix:')
print(M)


In [ ]:
# reorder transition matrix, from most to least common cluster
# diagonal elements monotonically decreasing

IDX_ordered = mh.reorder_cluster(IDX, M)

M = mh.make_transition_matrix(IDX_ordered)
print('\n transition matrix, ordered:')
print(M)

mh.image_M(M)


In [ ]:
# remove diagonal, and make markov, for display

print('transition matrix, diagonal elements removed, normalized (Markov)')

np.fill_diagonal(M, 0)  # happens inplace
M = mh.make_matrix_markov(M)

print(M)
mh.image_M(M, 1)
